In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import re

In [6]:
data_arr = []
with open('webservertrafficlog/access.log') as log:
    for line in log.readlines():
        match = re.match(r'(.+) - - \[(\d{2}\/\w{3}\/\d{4}:\d{2}:\d{2}:\d{2}) \+\d{4}\] \"(\w+) .+', line)
        if match:
            data_arr.append([match.group(1), match.group(2), match.group(3)])

data = pd.DataFrame(data_arr, columns=['ip_address', 'datetime', 'request_type'])

In [7]:
data

,ip_address,datetime,request_type
0,54.36.149.41,22/Jan/2019:03:56:14,GET
1,31.56.96.51,22/Jan/2019:03:56:16,GET
2,31.56.96.51,22/Jan/2019:03:56:16,GET
3,40.77.167.129,22/Jan/2019:03:56:17,GET
4,91.99.72.15,22/Jan/2019:03:56:17,GET
...,...,...,...
10365093,188.229.21.56,26/Jan/2019:20:29:13,GET
10365094,5.127.220.71,26/Jan/2019:20:29:13,GET
10365095,5.213.7.50,26/Jan/2019:20:29:13,GET
10365096,109.125.169.52,26/Jan/2019:20:29:13,GET


In [8]:
#Convert string datetime to Timestamp

data['datetime'] = data.datetime.apply(lambda x: x.replace('Jan', '01'))
data['datetime'] = pd.to_datetime(data.datetime, format='%d/%m/%Y:%H:%M:%S')


In [9]:
#add datetime columns to group by

data['day'] = data['datetime'].apply(lambda x: x.day)
data['hour'] = data['datetime'].apply(lambda x: x.hour)
data['minute'] = data['datetime'].apply(lambda x: x.minute)

In [10]:
#aggregate by minute and by hour

aggByMin = data.groupby(['day', 'hour', 'minute'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type'], axis=1)

aggByHour = data.groupby(['day', 'hour'], as_index=False).agg('count')\
  .rename({'ip_address': 'count'}, axis=1).drop(['datetime', 'request_type', 'minute'], axis=1)


In [11]:
data.to_csv('cleaned_log.csv')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10365098 entries, 0 to 10365097
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   ip_address    object        
 1   datetime      datetime64[ns]
 2   request_type  object        
 3   day           int64         
 4   hour          int64         
 5   minute        int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 474.5+ MB
